Script para tratamento dos dados para OD por geolocalização

Pacotes

In [ ]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import os

Carregamento dos arquivos



In [ ]:

#Arquivo bilhetagem
sbe_caminho = r"G:\.shortcut-targets-by-id\1KlHFtH5Jvo7J9AF_VBixQOXpFsNmZvhD\05.3. Pesquisa e Ciência dados - GEPDA\05.3.3 Data Science\05.3.3.1. Central de dados\05.3.1.2 OD\OD_ADAPTACAOBOTELHO\Dezembro_2023\20231206\exputiliz_2023-12-06.csv"
sbe = pd.read_csv(sbe_caminho, sep=";", encoding="latin1", dtype={'CODIGO_VEICULO': 'object', 'CODIGO_LINHA': 'object'}, low_memory=False)

In [ ]:
sbe['DATAHORA_UTILIZACAO'] = pd.to_datetime(sbe['DATAHORA_UTILIZACAO'])

In [ ]:
sbe['DATAHORA_UTILIZACAO'] = pd.to_datetime(sbe['DATAHORA_UTILIZACAO'])
sbe['HORARIO'] = sbe['DATAHORA_UTILIZACAO'].dt.strftime('%H:%M:%S')
sbe['DATA'] = sbe['DATAHORA_UTILIZACAO'].dt.strftime('%Y-%m-%d')
sbe['HORARIO'] = pd.to_datetime(sbe['HORARIO'])

In [ ]:
print(sbe[sbe['DATA'] == "2023-12-07"])

In [ ]:
valores_unicos = sbe['CODIGO_VEICULO'].unique()
valores_unicos

In [ ]:
sbe= sbe[
    ~(
        (sbe['TIPO_CARTAO'] == 'UNITÁRIO BRT') |(sbe['TIPO_CARTAO'] == 'CARTAO VIRTUAL')| (sbe['TIPO_CARTAO'] == 'OPERACIONAL BHTRANS') | (sbe['TIPO_CARTAO'] == 'ADMINISTRATIVO') | (sbe['TIPO_CARTAO'] == 'ACESSO TECNICO E OPERACIONAL') | (sbe['TIPO_CARTAO'] == 'GRATUIDADE SEM CARTAO') | (sbe['TIPO_CARTAO'] == 'AUXILIAR DE OPERAÇÕES')| (sbe['TIPO_CARTAO'] == 'RODOVIÁRIO METROPOLITANO'))]

In [ ]:
sbe

Associando o gps PING com o sbe

In [ ]:
# Aplicando múltiplos filtros em uma única operação
sbe_ping = sbe[
    ~(
        (sbe['NOME_OPERADORA'] == 'METRÔ BH') |
        (sbe['NOME_OPERADORA'] == 'CBTU') |
        sbe['NOME_OPERADORA'].str.startswith('ESTAÇÃO') |
        (sbe['NOME_OPERADORA'] == 'SINDPAUTRAS') |
        (sbe['CODIGO_LINHA'].str.startswith("AC")) |
        (sbe['CODIGO_LINHA'].str.startswith('CM')) |
        (sbe['CODIGO_LINHA'].str.startswith('PR')) |
        (sbe['CODIGO_LINHA'].str.startswith('SD')) |
        (sbe['CODIGO_VEICULO'].str.startswith('UC')) |
        (sbe['CODIGO_VEICULO'].str.startswith('UE'))
    )
]


In [ ]:
print(sbe_ping)

In [ ]:
def processar_linha(linha):
    # Remove caracteres indesejados e separa com base em ';'
    partes = linha.strip('<>;\n').split(';')
    # Cria um dicionário a partir dos valores separados
    return {parte.split('=')[0]: parte.split('=')[1] for parte in partes if parte}

diretorio_atual = os.getcwd()
caminho_pasta = os.path.join(diretorio_atual, "PING")

# Lista para armazenar os DataFrames
dfs = []

for arquivo in os.listdir(caminho_pasta):
    if arquivo.endswith(".csv"):
        caminho_completo = os.path.join(caminho_pasta, arquivo)
        with open(caminho_completo, 'r', encoding='latin-1') as f:
            linhas = f.readlines()[1:]  # Pula o cabeçalho
            dados = [processar_linha(linha) for linha in linhas]
            df = pd.DataFrame(dados)
            dfs.append(df)

gps = pd.concat(dfs, ignore_index=True)

In [ ]:
gps = gps.drop_duplicates().reset_index(drop=True)
gps["geometry"] = gps.apply(lambda row: Point(row["LG"], row["LT"]), axis=1)

gps_gdf = gpd.GeoDataFrame(gps, geometry="geometry", crs="EPSG:4326")
# gps_gdf = gps_gdf.to_crs("EPSG:31983")
gps_gdf["HR"] = pd.to_datetime(gps_gdf["HR"], format="%Y%m%d%H%M%S")
gps_gdf = gps_gdf.sort_values(by=["HR"]).reset_index(drop=True)


In [ ]:
gps_gdf.rename(columns={"HR": "DH", "NV": "CODIGO_VEICULO"}, inplace=True)

In [ ]:
gps_gdf['DH'] = pd.to_datetime(gps_gdf['DH'])
gps_gdf['HORARIO_GPS'] = gps_gdf['DH'].dt.strftime('%H:%M:%S')
gps_gdf['DATA_GPS'] = gps_gdf['DH'].dt.strftime('%Y-%m-%d')
gps_gdf = gps_gdf.dropna().reset_index(drop=True)

gps_gdf

In [ ]:
gps_gdf["CODIGO_VEICULO"]= gps_gdf["CODIGO_VEICULO"].astype("int64")
sbe_ping["CODIGO_VEICULO"]= sbe_ping["CODIGO_VEICULO"].astype("int64")

In [ ]:


# Convertendo as colunas de horário para datetime, se ainda não estiverem
gps_gdf['HORARIO_GPS'] = pd.to_datetime(gps_gdf['HORARIO_GPS'])
sbe_ping['HORARIO'] = pd.to_datetime(sbe_ping['HORARIO'])



# Fazendo o merge usando merge_asof para encontrar as linhas correspondentes em gps_gdf
merged = pd.merge_asof(sbe_ping.sort_values('HORARIO'),
                       gps_gdf.sort_values('HORARIO_GPS'),
                       left_on='HORARIO',
                       right_on='HORARIO_GPS',
                       by='CODIGO_VEICULO',
                       direction='backward',
                       tolerance=pd.Timedelta(seconds=40))


# Resultado contém as geometrias do gps_gdf correspondentes ao horário mais próximo anterior em sbe_final


Associando o metrô com a localização

In [ ]:
# Filtrando as linhas com NOME_OPERADORA igual a 'METRÔ BH'
sbe_metro = sbe[(sbe['NOME_OPERADORA'] == 'METRÔ BH') | (sbe['NOME_OPERADORA'] == 'CBTU')]



sbe_metro



In [ ]:
gps_geo_metro_caminho = r"G:\.shortcut-targets-by-id\1KlHFtH5Jvo7J9AF_VBixQOXpFsNmZvhD\05.3. Pesquisa e Ciência dados - GEPDA\05.3.3 Data Science\05.3.3.1. Central de dados\05.3.1.2 OD\OD_HEXAGONO\ESTACAO_METRO.csv"
gps_metro = pd.read_csv(gps_geo_metro_caminho, sep= ";", encoding= "latin1")
gps_metro = gps_metro.rename(columns={'CODIGO': 'CODIGO_VEICULO'})
# Converta a coluna 'CODIGO_LINHA' para string em ambos os DataFrames
sbe_metro['CODIGO_LINHA'] = sbe_metro['CODIGO_LINHA'].astype(str)
gps_metro['CODIGO_LINHA'] = gps_metro['CODIGO_LINHA'].astype(str)

# Realize um merge entre os DataFrames usando a coluna 'CODIGO_LINHA' como chave
sbe_metro_com_geometria = pd.merge(sbe_metro, gps_metro[['CODIGO_LINHA', 'geometry']], on='CODIGO_LINHA', how='left')

# O resultado será um novo DataFrame sbe_metro_com_geometria que incluirá a geometria do gps_metro



# # Exibindo o DataFrame resultante
sbe_metro_com_geometria.columns

Associando as estações com as localizações

In [ ]:

sbe_estacoes = sbe[sbe['CODIGO_LINHA'].str.startswith(('CM', 'PR', 'SD', 'AC', '1000', '1001', '1002', '2000', '2001', '2002', '3000','3001', '3002', '4001', '4000', '4002', '4003', '4004', '6000', '6001','7000', '7001'))]



sbe_estacoes


In [ ]:
sbe_est_caminho = r"G:\.shortcut-targets-by-id\1KlHFtH5Jvo7J9AF_VBixQOXpFsNmZvhD\05.3. Pesquisa e Ciência dados - GEPDA\05.3.3 Data Science\05.3.3.1. Central de dados\05.3.1.2 OD\OD_HEXAGONO\ESTACAO_ONIBUS V2.xlsx"
sbe_est = pd.read_excel(sbe_est_caminho)
sbe_est

In [ ]:
sbe_estacoes['CODIGO_LINHA'] = sbe_estacoes['CODIGO_LINHA'].astype(str)
sbe_est['CODIGO_LINHA'] = sbe_est['CODIGO_LINHA'].astype(str)
sbe_est_com_geometria = pd.merge(sbe_estacoes, sbe_est[['CODIGO_LINHA', 'geometry']], on='CODIGO_LINHA', how='left')
sbe_est_com_geometria

In [ ]:
sbe_est_com_geometria.columns

In [ ]:
merged.columns

In [ ]:
sbe_metro_com_geometria.columns

In [ ]:
sbe_unificado = pd.concat([sbe_est_com_geometria, sbe_metro_com_geometria, merged])


In [ ]:


# Convertendo 'DATAHORA_UTILIZACAO' para datetime
sbe_unificado['DATAHORA_UTILIZACAO'] = pd.to_datetime(sbe_unificado['DATAHORA_UTILIZACAO'])

# Ordenando o DataFrame
sbe_sorted = sbe_unificado.sort_values(['CARTAO_USUARIO', 'DATAHORA_UTILIZACAO'])

# Calculando a diferença de tempo entre registros consecutivos em minutos
sbe_sorted['time_diff'] = sbe_sorted.groupby('CARTAO_USUARIO')['DATAHORA_UTILIZACAO'].diff().dt.total_seconds() / 60

# Filtrar validações com intervalos menores que 5 minutos
sbe_filtered = sbe_sorted[(sbe_sorted['time_diff'] >= 5) | (sbe_sorted.groupby('CARTAO_USUARIO').cumcount() == 0)]

# Selecionar apenas as primeiras validações de cada cartão
primeiras_val = sbe_filtered[sbe_filtered.groupby('CARTAO_USUARIO').cumcount() == 0]

# Separar as validações com intervalos maiores e menores que 90 minutos
val_maiores_90 = sbe_filtered[sbe_filtered['time_diff'] >= 90]
val_menores_90 = sbe_filtered[sbe_filtered['time_diff'] < 90]

sbe_final = pd.concat([primeiras_val, val_maiores_90, val_menores_90])


In [ ]:
print(sbe_final[sbe_final['CARTAO_USUARIO'] == 2310000029300])

In [ ]:

# Supondo que merged é o seu DataFrame
# Garantir que os dados estão ordenados corretamente
sbe_final = sbe_final.sort_values(['CARTAO_USUARIO', 'DATAHORA_UTILIZACAO'])

# Deslocando os dados para obter a próxima validação
sbe_final['DESTINO_HORARIO'] = sbe_final.groupby('CARTAO_USUARIO')['DATAHORA_UTILIZACAO'].shift(-1)
sbe_final['DESTINO_GEOMETRY'] = sbe_final.groupby('CARTAO_USUARIO')['geometry'].shift(-1)

# Para a última linha de cada grupo, definir o destino como a primeira origem do grupo
for _, group in sbe_final.groupby('CARTAO_USUARIO'):
    if len(group) > 1:
        last_index = group.index[-1]
        first_index = group.index[0]
        sbe_final.at[last_index, 'DESTINO_HORARIO'] = group.at[first_index, 'DATAHORA_UTILIZACAO']
        sbe_final.at[last_index, 'DESTINO_GEOMETRY'] = group.at[first_index, 'geometry']


# Exibindo o resultado
print(sbe_final[sbe_final['CARTAO_USUARIO'] == 6850001298748])


Juntando todos os dataframes com as localizações e analisando as diferenças entre 90 minutos de validação

In [ ]:
print(sbe_final.columns)

Tratamento dos dados

Função de ajuste dos dados do gps

In [ ]:
# Criando a lista de colunas a serem excluídas
colunas_excluir = ['NOME_OPERADORA', 'Unnamed: 7', 'EV', 'DH', 'DESTINO_HORARIO']

# Usando a lista para excluir as colunas do DataFrame
od = sbe_final.drop(colunas_excluir, axis=1)
od.reset_index(drop=True, inplace=True)
# Exibindo o DataFrame resultante
print(od)


In [ ]:
# Renomeando a coluna 'geometry' para 'ORIGIN_GEOMETRY'
od = od.rename(columns={'geometry': 'ORIGIN_GEOMETRY'})

# Exibindo o DataFrame resultante
print(od.columns)


In [ ]:
print(od[od["CARTAO_USUARIO"] == 6850001298748])

In [ ]:
od['HORA_ORIGEM'] = od['DATAHORA_UTILIZACAO'].dt.hour

In [ ]:
print(od[od["CARTAO_USUARIO"] == 6850001298748])

In [ ]:
import h3
from shapely.geometry import Point

In [ ]:


# Supondo que 'od' é o seu DataFrame e 'ORIGIN_GEOMETRY' e 'DESTINO_GEOMETRY' são do tipo Point

# Extraindo latitude e longitude para origem e destino
od['origin_lat'] = od['ORIGIN_GEOMETRY'].apply(lambda point: point.y if isinstance(point, Point) else None)
od['origin_lon'] = od['ORIGIN_GEOMETRY'].apply(lambda point: point.x if isinstance(point, Point) else None)
od['destino_lat'] = od['DESTINO_GEOMETRY'].apply(lambda point: point.y if isinstance(point, Point) else None)
od['destino_lon'] = od['DESTINO_GEOMETRY'].apply(lambda point: point.x if isinstance(point, Point) else None)

# Definindo o nível de resolução do H3, por exemplo, 7
resolution = 9

# Calculando o índice H3 para origem e destino
od['h3_origin'] = od.apply(lambda row: h3.geo_to_h3(row['origin_lat'], row['origin_lon'], resolution) if pd.notnull(row['origin_lat']) and pd.notnull(row['origin_lon']) else None, axis=1)
od['h3_destino'] = od.apply(lambda row: h3.geo_to_h3(row['destino_lat'], row['destino_lon'], resolution) if pd.notnull(row['destino_lat']) and pd.notnull(row['destino_lon']) else None, axis=1)

# Exibindo o DataFrame resultante
print(od[['ORIGIN_GEOMETRY', 'h3_origin', 'DESTINO_GEOMETRY', 'h3_destino']])



In [ ]:
od_analise_linha = od[(od['h3_origin'].notna()) | (od['h3_destino'].notna())]
od_analise_linha = od_analise_linha.groupby('CODIGO_LINHA').size().reset_index(name='Origem e destinos')

# Exibindo o DataFrame resultante
print(od_analise_linha)

In [ ]:
od_analise_linha_melhor = od[(od['h3_origin'].notna()) & (od['h3_destino'].notna())]
od_analise_linha_melhor = od_analise_linha_melhor.groupby('CODIGO_LINHA').size().reset_index(name='Origem e destinos')

# Exibindo o DataFrame resultante
print(od_analise_linha_melhor)

In [ ]:
od_analise_linha

In [ ]:
print(od[od['CODIGO_LINHA']== "67"])

In [ ]:
od['CARTAO_USUARIO'] = "ID" + od["CARTAO_USUARIO"].astype(str)


In [ ]:
od

In [ ]:
od.to_csv(r'g:\.shortcut-targets-by-id\1KlHFtH5Jvo7J9AF_VBixQOXpFsNmZvhD\05.3. Pesquisa e Ciência dados - GEPDA\05.3.3 Data Science\05.3.3.1. Central de dados\05.3.1.2 OD\OD_HEXAGONO\od.csv', sep =';')

In [ ]:
mco = r"G:\.shortcut-targets-by-id\1KlHFtH5Jvo7J9AF_VBixQOXpFsNmZvhD\05.3. Pesquisa e Ciência dados - GEPDA\05.3.3 Data Science\05.3.3.1. Central de dados\05.3.1.2 OD\OD_ADAPTACAOBOTELHO\Dezembro_2023\20231206\relopemapadecontroleoperacional.csv"

In [ ]:
mco_analise_linha = pd.read_csv(mco, sep = ";", encoding= "latin1")
# Renomeando a coluna 'Código Externo Linha' para 'CODIGO_LINHA'
mco_analise_linha = mco_analise_linha.rename(columns={'Código Externo Linha': 'CODIGO_LINHA'})
mco_analise_linha.columns

In [ ]:
# Agrupando e contando
od_analise_linha = od[(od['h3_origin'].notna()) | (od['h3_destino'].notna())].groupby('CODIGO_LINHA').size()


# Convertendo para DataFrame e renomeando a coluna
od_analise_linha = pd.DataFrame(od_analise_linha, columns=['Contagem_ids_OD'])

# Exibindo o resultado
print(od_analise_linha)


In [ ]:
# Agrupando por 'Código Externo Linha' e somando as colunas 'Passageiros' e 'Inteiras'
agrupado_soma = mco_analise_linha.groupby('CODIGO_LINHA')[['Passageiros', 'Inteiras']].sum()

# Calculando a divisão da coluna 'Inteiras' pela coluna 'Passageiros'
agrupado_soma['Proporcao_Inteiras'] = agrupado_soma['Inteiras'].div(agrupado_soma['Passageiros'].replace(0, 1))



# Exibindo o DataFrame resultante
print(agrupado_soma)



In [ ]:
# Realizando o merge entre agrupado_soma e od_analise_linha
analise_final = pd.merge(od_analise_linha, agrupado_soma, on='CODIGO_LINHA')

# Exibindo o DataFrame resultante
print(analise_final)
analise_final.to_excel(r'G:\.shortcut-targets-by-id\1KlHFtH5Jvo7J9AF_VBixQOXpFsNmZvhD\05.3. Pesquisa e Ciência dados - GEPDA\05.3.3 Data Science\05.3.3.1. Central de dados\analise_linha.xlsx')

In [ ]:
# Realizando o merge entre agrupado_soma e od_analise_linha
analise_final_melhorcenario = pd.merge(od_analise_linha_melhor, agrupado_soma, on='CODIGO_LINHA')

# Exibindo o DataFrame resultante
print(analise_final)
analise_final_melhorcenario.to_excel(r"G:\.shortcut-targets-by-id\1KlHFtH5Jvo7J9AF_VBixQOXpFsNmZvhD\05.3. Pesquisa e Ciência dados - GEPDA\05.3.3 Data Science\05.3.3.1. Central de dados\05.3.1.2 OD\OD_HEXAGONO\geral.xlsx")

In [ ]:


# Exemplo de identificador H3
h3_index = '89440e4ecb7ffff'

# Obter as coordenadas dos vértices do hexágono
hex_boundary = h3.h3_to_geo_boundary(h3_index, geo_json=True)

# Converter as coordenadas para um formato adequado para folium (latitude, longitude)
hex_coords = [[coord[1], coord[0]] for coord in hex_boundary]

# Criar um mapa folium centrado no hexágono
mapa = folium.Map(location=hex_coords[0], zoom_start=12)

# Adicionar o polígono do hexágono ao mapa
folium.Polygon(locations=hex_coords, color='blue', fill=True, fill_color='blue').add_to(mapa)

# Mostrar o mapa
mapa.save(r'g:\.shortcut-targets-by-id\1KlHFtH5Jvo7J9AF_VBixQOXpFsNmZvhD\05.3. Pesquisa e Ciência dados - GEPDA\05.3.3 Data Science\05.3.3.1. Central de dados\05.3.1.2 OD\OD_HEXAGONO\heatmap.html')

In [ ]:
hora_especifica = 10
df_filtrado = od[od['HORA_ORIGEM'] == hora_especifica]
df_filtrado

In [ ]:
from folium.plugins import HeatMap
import folium
import h3

# Contar os destinos baseados em índices H3
contagem_destinos = df_filtrado['h3_destino'].value_counts()

# Converter a contagem de destinos para um formato adequado para o mapa de calor
pontos_mapa_calor = [(h3.h3_to_geo(h)[0], h3.h3_to_geo(h)[1], count) for h, count in contagem_destinos.items()]

# Calcular o centro do mapa como a média das localizações dos destinos
latitudes = [h3.h3_to_geo(h)[0] for h in contagem_destinos.keys()]
longitudes = [h3.h3_to_geo(h)[1] for h in contagem_destinos.keys()]
centro_mapa = [sum(latitudes) / len(latitudes), sum(longitudes) / len(longitudes)]

# Criar um mapa centrado nesse ponto
mapa = folium.Map(location=centro_mapa, zoom_start=12)

# Adicionar o mapa de calor ao mapa com um raio menor
HeatMap(pontos_mapa_calor, radius=10).add_to(mapa)  # Ajuste o valor de 'radius' conforme necessário


In [ ]:
mapa.save(r'g:\.shortcut-targets-by-id\1KlHFtH5Jvo7J9AF_VBixQOXpFsNmZvhD\05.3. Pesquisa e Ciência dados - GEPDA\05.3.3 Data Science\05.3.3.1. Central de dados\heatmap.html')